In [2]:
%matplotlib qt

In [3]:
import matplotlib.pyplot as plt
from matplotlib import cm
from astropy.io import fits
import os
import numpy as np
from scipy.interpolate import CubicSpline

In [4]:
def extract_spectrum_within_range(wavelengths, flux, start_wavelength, end_wavelength):
    """
    Extract wavelength and flux values within a given range.

    Parameters:
    - wavelengths: List or array of wavelength values.
    - flux: List or array of flux values.
    - start_wavelength: Lower bound of the wavelength range.
    - end_wavelength: Upper bound of the wavelength range.

    Returns:
    - extracted_wavelengths: Wavelength values within the specified range.
    - extracted_flux: Flux values corresponding to the selected wavelengths.
    """

    # Find indices corresponding to the specified wavelength range
    indices = np.where((wavelengths >= start_wavelength) & (wavelengths <= end_wavelength))[0]

    # Extract wavelength and flux values within the range
    extracted_wavelengths = wavelengths[indices]
    extracted_flux = flux[indices]

    return extracted_wavelengths, extracted_flux

In [5]:
def divide_colormap(num_parts, colormap_name='rainbow'):
    cmap = plt.get_cmap(colormap_name)
    num_colors = cmap.N
    colors = [cmap(i / num_colors) for i in range(0, num_colors, num_colors // num_parts)]
    return colors



def import_spectra(object: str):

    # Path to root folder
    folder_path = f"../Spectra/{object}/"

    # Get a list of all files in the folder
    all_files = os.listdir(folder_path)

    # Filter files that start with "ADP"
    adp_files = [file for file in all_files if file.startswith("ADP")]

    # Gather the spectra
    spectra = []
    for file in adp_files:
        data = fits.getdata(folder_path + file)
        wavelength = data['WAVE'][0]
        flux = data['FLUX'][0]
        spectra.append((wavelength, flux))

    return spectra



def plot_spectrum(object: str):

    # Import spectra
    spectra = import_spectra(object)

    # Spectral lines and wavelengths in angstrom
    spectral_lines = [
        (4026, "HeI+II 4026"),
        (4200, "HeII 4200"),
        (4686, "HeII 4686"),
        (4634, "NIII 4634-40-42 (emission)"),
        (4144, "HeI 4144"),
        (4388, "HeI 4388"),
        (4542, "HeII 4542"),
        (4552, "SiII 4552"),
        (4686, "HeII 4686"),
        (4861, "Hb 4861"),
        (5016, "HeI 5016"),
        (5876, "HeI 5876"),
        (5890, "NaI 5890"),
        (5896, "NaI 5896"),
        (6527, "HeII 6527"),
        (6563, "Ha 6563")
    ]

    spectral_lines = [
        (4026, "He I + II 4026"),
        (4200, "He II 4200"),
        (4634, "N III 4634-40-42 (emission)"),
        (4686, "He II 4686"),
        (4144, "He I 4144"),
        (4388, "He I 4388"),
        (4541, "He II 4541"),
        (4552, "Si II 4552"),
        (4686, "He II 4686"),
        (4861, "Hb 4861"),
        (5016, "He I 5016"),
        (5876, "He I 5876"),
        (5890, "Na I 5890"),
        (5896, "Na I 5896"),
        (6527, "He II 6527"),
        (6563, "Ha 6563"),
        (4471, "He I 4471"),
        (4058, "N IV 4058"),
        (4116, "Si IV 4116"),
        (4097, "N III 4097"),
        (4504, "Si IV 4686-4504"),
        (4713, "He I 4713"),
        (4187, "C III 4187"),
        (4121, "He I 4121"),
        (3995, "N II 3995"),
        (4350, "O II 4350"),
        (4128, "Si I 4128-30"),
        (4481, "Mg II 4481"),
        (4233, "Fe II 4233"),
    ]

    colors = divide_colormap(len(spectral_lines))

    # Plot the spectrum
    plt.figure(figsize=(10, 5))
    for spectrum in spectra:
        plt.plot(spectrum[0], spectrum[1])

    # Plot wavelengths of spectral lines
    for wavelength, label in spectral_lines:
        plt.vlines(wavelength, -1000, 1000, label=label, color=colors[spectral_lines.index((wavelength, label))])

    plt.xlabel('Wavelength (Angstroms)')
    plt.ylabel('Flux')
    plt.title(f'Spectrum {object}')
    # plt.ylim(0, 1.2)
    plt.legend()
    plt.show()

    return

# Example
plot_spectrum('4U1538_52')

In [6]:
spectra = import_spectra('4U1700_37')
line_wav, line_flux = extract_spectrum_within_range(spectra[3][0], spectra[3][1], 4021.8, 4027.8)

# Only background
bg1_wav, bg1_flux =  extract_spectrum_within_range(spectra[3][0], spectra[3][1], 4010, 4020)
bg2_wav, bg2_flux =  extract_spectrum_within_range(spectra[3][0], spectra[3][1], 4030, 4040)
bg_wav = np.concatenate([bg1_wav, bg2_wav])
bg_flux = np.concatenate([bg1_flux, bg2_flux])

plt.plot(line_wav, line_flux)
plt.plot(bg_wav, bg_flux)

# Fit a 2nd degree polynomial
coefficients = np.polyfit(bg_wav, bg_flux, 2)
polynomial = np.poly1d(coefficients)

# Generate points for the fitted polynomial curve
bg_wav_fit = np.linspace(min(bg_wav), max(bg_wav), 100)
bg_flux_fit = polynomial(bg_wav_fit)

# Plot the original data points and the fitted polynomial curve
plt.scatter(bg_wav, bg_flux, label='Original Data')
plt.plot(bg_wav_fit, bg_flux_fit, label='Fitted Polynomial (2nd degree)', color='red')

# Add labels and legend
plt.xlabel('bg_wav')
plt.ylabel('bg_flux')
plt.legend()

# Show the plot
plt.show()

In [7]:
# Calculate EW
continuum = polynomial(line_wav)
dlambda = line_wav[1] - line_wav[0]
EW = 0
area = 0
for Fc, Fs in zip(continuum, line_flux):
    EW += (1 - Fs / Fc) * dlambda
print(EW)

0.4049989533042062


In [8]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# Generate example data (replace this with your own spectrum data)
wavelength = spectra[0][0]
flux = spectra[0][1]  # Replace with your actual flux data

# Define a function for the polynomial fit (adjust the degree as needed)
def continuum_fit(x, *params):
    return np.polyval(params, x)

# Initial guess for polynomial coefficients
initial_guess = np.ones(2)

# Perform the fit using curve_fit from scipy
fit_params, covariance = curve_fit(continuum_fit, wavelength, flux, p0=initial_guess)

# Generate the continuum using the fitted parameters
continuum = continuum_fit(wavelength, *fit_params)

# Plot the original spectrum and the fitted continuum
plt.plot(wavelength, flux, label='Original Spectrum')
plt.plot(wavelength, continuum, label='Fitted Continuum', linestyle='--', color='red')
plt.legend()
plt.xlabel('Wavelength')
plt.ylabel('Flux')
plt.title('Continuum Fitting Example')
plt.show()


In [9]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter

# Generate example data (replace this with your own spectrum data)
wavelength = spectra[0][0]
flux = spectra[0][1]  # Replace with your actual flux data

# Choose the window size and polynomial order for Savitzky-Golay filter
window_size = 15
poly_order = 2

# Apply Savitzky-Golay filtering to the flux data
smoothed_flux = savgol_filter(flux, window_size, poly_order)

# Plot the original spectrum and the filtered spectrum
plt.plot(wavelength, flux, label='Original Spectrum')
plt.plot(wavelength, smoothed_flux, label='Filtered Spectrum', linestyle='--', color='red')
plt.legend()
plt.xlabel('Wavelength')
plt.ylabel('Flux')
plt.title('Spectral Line Filtering Example (Savitzky-Golay)')
plt.show()


In [10]:
import numpy as np
import matplotlib.pyplot as plt

# Assuming you have your spectrum data in arrays x and y
# and you have already fitted a linear model (e.g., using np.polyfit)

# Example spectrum data
x = spectra[3][0]
y = spectra[3][1]

# Fit a linear model (in this case, a first-degree polynomial)
coefficients = np.polyfit(x, y, 1)
linear_fit = np.polyval(coefficients, x)

# Calculate the difference between actual data and linear fit
residuals = y - linear_fit

# Create a mask for points under the linear fit
mask_under_fit = residuals < 0

# Apply the mask to your data
masked_x = x[mask_under_fit]
masked_y = y[mask_under_fit]

# Plotting for visualization
plt.scatter(x, y, label='Actual Data')
plt.plot(x, linear_fit, label='Linear Fit', color='red')
plt.scatter(masked_x, masked_y, label='Points under Linear Fit', color='green')
plt.legend()
plt.show()


In [11]:
mask_above_fit = residuals > 0
filtered_spectrum_wav = x[mask_above_fit]
filtered_spectrum_flux = y[mask_above_fit]
# plt.plot(filtered_spectrum_wav, filtered_spectrum_flux)

# Choose the window size and polynomial order for Savitzky-Golay filter
window_size = 1000
poly_order = 0

smoothed_flux = savgol_filter(filtered_spectrum_flux, window_size, poly_order)
plt.plot(spectra[3][0], spectra[3][1])
plt.plot(filtered_spectrum_wav, smoothed_flux, label='Filtered Spectrum', linestyle='--', color='red')

In [12]:
# Fit a linear model (in this case, a first-degree polynomial)
coefficients = np.polyfit(filtered_spectrum_wav, smoothed_flux, 3)
linear_fit = np.polyval(coefficients, filtered_spectrum_wav)
plt.plot(spectra[3][0], spectra[3][1])
plt.plot(filtered_spectrum_wav, linear_fit)


In [17]:
# Path to root folder
folder_path = f"../Spectra/{'4U1538_52'}/"

# Get a list of all files in the folder
all_files = os.listdir(folder_path)

# Filter files that start with "ADP"
adp_files = [file for file in all_files if file.startswith("ADP")]

data = fits.getdata(folder_path + adp_files[0])

In [32]:
plt.plot(data['WAVE'][0], data['FLUX'][0])
plt.ylim(-50, 50)

(-50.0, 50.0)